# **HuggingFace Models**

## installation
https://python.langchain.com/docs/integrations/chat/huggingface/

In [1]:
from google.colab import userdata
import getpass
import os

# https://huggingface.co/settings/tokens
os.environ["HUGGINGFACEHUB_API_TOKEN"]=userdata.get('huggingToken')

if "HUGGINGFACEHUB_API_TOKEN" not in os.environ:
    os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass.getpass("Enter your Hugging Face API key: ")

In [2]:
%pip install --upgrade --quiet langchain-huggingface

## Basic FewShots

In [3]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_core.messages import SystemMessage, AIMessage, HumanMessage
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts.few_shot import FewShotPromptTemplate

repo_id = "microsoft/Phi-3-mini-4k-instruct"

llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
)

chat = ChatHuggingFace(llm=llm, verbose=True)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
demo_message = """
Country name: Egypt | Captital name: Cairo
Country name: France | Captital name: Paris
Country name: Italy | Captital name: Rome
Country name: Japan | Captital name: Tokyo
Country name: USA | Captital name: Washington D.C.
Country name: Morocco | Captital name:
""".strip()

ai_response = chat.invoke(demo_message)
ai_response.content


'Egypt - Cairo\nFrance - Paris\nItaly - Rome\nJapan - Tokyo\nUSA - Washington D.C.\nMorocco - Rabat'

In [5]:
demo_template = PromptTemplate(
    template="Country name: {country} | Captital name: {capital}",
    input_variables=["country", "capital"],
)

examples = [
    {"country": "Egypt", "capital": "Cairo"},
    {"country": "France", "capital": "Paris"},
    {"country": "Italy", "capital": "Rome"},
]

prompt = FewShotPromptTemplate(
    example_prompt=demo_template,
    examples=examples,
    suffix="Country name: {country} | Captital name:",
    input_variables=["country"],
)

user_input = "Japan"

message=prompt.format(country=user_input)

# print(message)

ai_response = chat.invoke(message)
ai_response.content



'Country name: Japan | Captial name: Tokyo'

# Fewshot Learning

In [6]:
problem_prompt="When I was seven, my sister was twice my age. Now I am seventy years old, how old can my sister be?"

ai_response = chat.invoke(problem_prompt)
ai_response.content

'When you were seven, your sister was twice your age, meaning she was 14 years old. The difference in age between you and your sister would have remained constant. That means your sister was 14 - 7 = 7 years older than you. Now that you are seventy, your sister would be 70 + 7 = 77 years old.'